<a href="https://colab.research.google.com/github/Zurkon35/DSCI401/blob/main/Midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
employee = pd.read_csv("https://raw.githubusercontent.com/gjm112/DSCI401/main/data/Employee.csv")
stops = pd.read_csv("https://raw.githubusercontent.com/gjm112/DSCI401/main/data/CTA_List_of_L_Stops.csv")
entries = pd.read_csv("https://raw.githubusercontent.com/gjm112/DSCI401/main/data/CTA_entries.csv")

In [ ]:
# Question 1a

female = employee[employee['Gender'] == 'Female'].groupby('Education')
male = employee[employee['Gender'] == 'Male'].groupby('Education')

print(female)
print(male)

In [ ]:
# Question 1b

hire = employee.groupby(["City", "JoiningYear"]).size().reset_index(name='hires')
hire = hire.sort_values(by=["City", "JoiningYear"])

g = hire.plot(x = "JoiningYear", y = "hires")
g.set_xlabel("Year Joined")
g.set_ylabel("Total Hires")
plt.show()

# When I added color = "City" I kept getting an error

In [ ]:
# Question 1c

gg = sns.FacetGrid(employee, col = "City", row = "Education")
gg.map(sns.boxplot, "Age", "City")


In [ ]:
# Question 1d

year_ratio = employee.groupby('Education') \
    .apply(lambda x: x.assign(n = len(x))) \
    .groupby(['JoiningYear'], as_index=False) \
    .apply(lambda x: x.assign(ratio = len(x)/x['n'].iloc[0])) \
    .reset_index(drop=True) \
    .groupby('JoiningYear', as_index=False, group_keys=False) \
    .apply(lambda x: x)


print(year_ratio)

In [ ]:
# Question 1d cont...

city_ratio = employee.groupby('Education') \
    .apply(lambda x: x.assign(n = len(x))) \
    .groupby(['City'], as_index=False) \
    .apply(lambda x: x.assign(ratio = len(x)/x['n'].iloc[0])) \
    .reset_index(drop=True) \
    .groupby('City', as_index=False, group_keys=False) \
    .apply(lambda x: x)

print(city_ratio)

In [ ]:
# Question 2a

stops['RED'] = stops['RED'].astype(int)
red = stops[['RED', 'MAP_ID']].loc[stops['RED'] == 1]
entries_1 = entries.drop([2])
entries_red = red.merge(entries_1, left_on='MAP_ID', right_on='station_id')

print(entries_red)


In [ ]:
# Question 2b

entries_red['date'] = pd.to_datetime(entries_red['date'], format='%m/%d/%Y')
entries_red['Year'] = entries_red['date'].dt.year
entries_red['Month'] = entries_red['date'].dt.month

avg_rides_station = entries_red.groupby('station_id').agg(Avg_rides_day=('rides', 'mean'), total_rides=('rides', 'sum')).reset_index().sort_values(by='Avg_rides_day', ascending=False)
avg_rides_month = entries_red.groupby(['station_id', 'Year', 'Month']).agg(avg_rides_m=('rides', 'mean')).reset_index()

print(avg_rides_station)
print(avg_rides_month)

In [ ]:
# Quesiton 2c

new_avg_month = avg_rides_month.merge(avg_rides_station, on="station_id")
new_avg_month["ratio"] = new_avg_month["avg_rides_m"] / new_avg_month["Avg_rides_day"]

top_10 = new_avg_month.sort_values(by="ratio", ascending=False).head(10)

print(top_10)

In [ ]:
# Question 2d

sd_top10 = new_avg_month.groupby(['station_id', 'Month', 'ratio']).agg(sd_month=('avg_rides_m', np.std)).reset_index()
sd_top10 = sd_top10.sort_values(by='sd_month', ascending=False).head(5)

ggg = sns.lineplot(data=sd_top10, x='Month', y='ratio', hue='station_id')
ggg.set(xlabel='Month', ylabel='Ratio')

print(ggg)

# I am not quite sure how to fix this graph